In [5]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def stockReturns(priceDF):
    
    # Method 1: Compute daily returns by parsing the Data Frame as numpy matrix (Fastest)
    compTickers = priceDF.columns[1: ]    
    priceMat = priceDF.loc[ : , compTickers].as_matrix()    
    diffMat = (priceMat[1: ] - priceMat[ :-1]) / priceMat[ :-1]
    
    return pd.DataFrame(data = diffMat, index = priceDF.index[1: ], columns = compTickers)

def calCorrelations(dailyReturn):
    
    # Method 1: Pandas built-in function to calculate pairwise correlation (Fastest)
#    return dailyReturn.corr()
    
    # Method 2: Manual calculation of pairwise correlation using numpy matrix (Faster)
    col = dailyReturn.columns
    ncol = len(col)
    corrMat = np.identity(ncol)
    
    G = nx.Graph()
    G.add_nodes_from(col.values)
        
    n = len(dailyReturn)
    for i in range(0, ncol):
        for j in range(i + 1, ncol):
            x = dailyReturn[col[i]]
            y = dailyReturn[col[j]]
            xsum = sum(x)
            ysum = sum(y)
            corrMat[i][j] = (n * sum(x * y) - xsum * ysum) / (np.sqrt(n * sum(x**2) - xsum**2) * np.sqrt(n * sum(y**2) - ysum**2))
            corrMat[j][i] = corrMat[i][j]
            G.add_edge(col[i], col[j], weight = corrMat[i][j])
        
    return pd.DataFrame(data = corrMat, index = col, columns = col), G

def stockClustering(graph, k):
            
    # Method 1: Clustering using dictionary and list
    sortedEdges = sorted(graph.edges(data = True), key = lambda edge: edge[2]['weight'], reverse = True) # O(e log e), e = number of edges = n(n - 1)/2 for dense graph
    
    listSets = dict()    
    for node in graph.nodes(): # O(n), n = number of companies
        listSets[node] = {node}
        
    for i in range(0, k): # Execute k times
        (a, b, data) = sortedEdges[i]
        mergedSet = listSets[a].union(listSets[b]) # O(len(set(a)) + len(set(b))) = O(n) in worst case since the biggest possible set is the set with all n companies
        for node in mergedSet: # O(n)
            listSets[node] = mergedSet
    # The whole chunk above is O(kn + kn) = O(kn)
            
    resultSets = []
    for nodeSet in listSets.values(): # O(n)
        if nodeSet not in resultSets:
            resultSets.append(nodeSet)
        
    return resultSets

priceDF = pd.read_csv('SP_500_close_2015.csv')
firmDF = pd.read_csv('SP_500_firms.csv', index_col = 0)

dailyReturn = stockReturns(priceDF)

corrDF, corrGraph = calCorrelations(dailyReturn)

cluster = stockClustering(corrGraph, 1000)

def getStockDetails(ticker, clusterSets, firmDF):
    
    # if the ticker given is in a single set cluster
    if {ticker} in clusterSets:
        return firmDF.loc[ticker, :]
    else:
        for i in range(len(clusterSets)):
            if ticker in clusterSets[i]:
                return firmDF.loc[clusterSets[i], :].sort_values('Sector')
            


## Average Linkage Clustering 

The first alternative clustering algorithm we will be looking at is Average Linkage Clustering. Both Average-Linkage Clustering and Single-Linkage Clustering are subtypes of Agglomerative Hierarchical Clustering. The difference between these two clustering methods is the definition of 'shortest distance'. Note that in our earlier example, correlation between stocks was used to represent similarities. We can think of correlation as the inverse of 'distance' between the nodes.  

In single-linkage clustering, the distance between two clusters is determined by a single element pair (one from each cluster) which are closest to each other. The algorithm always chooses the shortest of these links while merging two clusters. While merging cluster, single-linkage clustering disregards the distance to all other elements in the cluster. Therefore, it tends to produce large cluster where ceertain pairs of elements are dissimilar to each other.  

However, in average-linkage clustering, the distance between two clusters is defined as the average of all distances between pairs of objects in the two clusters. This helps to prevent the so-called chaining phenomenon experienced by single-linkage clustering, where many of the elements in each cluster may be very dissimilar.  

### Python Code  

The following section includes the Python code in performing average-linkage clustering using scikit-learn. Since the scikit-learn function requires a distance matrix, we will need to convert the correlation matrix into distance matrix.    

Convert the correlation matrix into distance matrix. Correlation is within the interval [-1, 1] and a higher value reflects greater similarity. We'll need to transform correlation value into distance, hence we get the absolute value of (correlation - 1). The resulting value will be a distance within interval [0, 2] and a higher value reflects lower similarity.  


In [ ]:
from sklearn.cluster import AgglomerativeClustering

def stockClusteringAgglomerative(corrDF, num_clusters):
    
    # Store column names as company's ticker symbol
    compTickers = corrDF.columns
    
    # Transform correlation matrix into distance matrix
    affinityMatrix = abs(corrDF.as_matrix() - 1)
    
    # Run agglomerative clustering with average-linkage with precomputed distance matrix
    model = AgglomerativeClustering(linkage = 'average', affinity = 'precomputed', n_clusters = num_clusters)
    aggFit = model.fit(affinityMatrix)
    
    resultSets = [set()] * num_clusters
    i = 0
    
    # Return the clustering results in the form of "list of sets", each set representing a cluster
    for l in aggFit.labels_:
        resultSets[l] = resultSets[l].union({compTickers[i]})
        i += 1
        
    return resultSets

# Call cluster-linkage clustering using correlation DF, and num_cluster = number of clusters produced earlier
clusterAggAveLinkage = stockClusteringAgglomerative(corrDF, len(cluster))


## K-Means Clustering 

The second clustering algorithm we use is K-Means Clustering. To use K-Means clustering, we need to define the "features" of each node. Correlation only provides the distance between each node and hence is not sufficient for K-Means clustering. In this case, we define the features to be daily price change (in percentage) of each individual stock. This information is contained in the daily return matrix where each column contains the daily price change in terms of percentage.  

K-Means clustering attempts to fit K number of centroids to the dataset such that the total sum of distance from each node to its corresponding centroid is minimised. One characterisitc of K-Means clustering is that it tends to separate data into clusters of roughly equal size. In other words, it assumes the data distribution to be separable with K spherical clusters. This might not work well on certain data where this assumption does not hold true.  

Lastly, since K-Means clustering assumes the data to be separable using K spherical clusters of roughly equal size, the choice of K value is key. A bad choice of K can lead to bad results. We will explore this in later section.  

### Python Code  

The following section includes the Python code in performing K-Means clustering using scikit-learn. As mentioned above, K-Means clustering function requires a list of vectors defining the "features" of each stock. We will use the daily return for K-Means clustering algorithm.  

In [ ]:
from sklearn.cluster import KMeans

def stockClusteringKMeans(dailyReturn, num_clusters):
    
    # Store column names as company's ticker symbol
    compTickers = dailyReturn.columns
    
    # Transpose the daily return matrix so that each row contains all the price changes of a particular stock
    dailyReturnArray = dailyReturn.as_matrix().transpose()
    
    # Run K-Means clustering with random centroid initialisation
    kmeans = KMeans(n_clusters = num_clusters, random_state = 0).fit(dailyReturnArray)
    
    resultSets = [set()] * num_clusters
    i = 0
    
    # Return the clustering results in the form of "list of sets", each set representing a cluster
    for l in kmeans.labels_:
        resultSets[l] = resultSets[l].union({compTickers[i]})
        i += 1
    
    return resultSets

clusterKMeans = stockClusteringKMeans(dailyReturn, len(cluster))


In [6]:
getStockDetails('GS', cluster, firmDF)

,Name,Sector
Symbol,,
SWK,Stanley Black & Decker,Consumer Discretionary
SNA,Snap-On Inc.,Consumer Discretionary
BAC,Bank of America Corp,Financials
PBCT,People's United Financial,Financials
TROW,T. Rowe Price Group,Financials
PNC,PNC Financial Services,Financials
LM,Legg Mason,Financials
AFL,AFLAC Inc,Financials
AMP,Ameriprise Financial,Financials


In [7]:
getStockDetails('GS', clusterAggAveLinkage, firmDF)

,Name,Sector
Symbol,,
LNC,Lincoln National,Financials
ETFC,E*Trade,Financials
PRU,Prudential Financial,Financials
JPM,JPMorgan Chase & Co.,Financials
HBAN,Huntington Bancshares,Financials
BBT,BB&T Corporation,Financials
GS,Goldman Sachs Group,Financials
KEY,KeyCorp,Financials
STI,SunTrust Banks,Financials


In [8]:
getStockDetails('GS', clusterKMeans, firmDF)

,Name,Sector
Symbol,,
AMP,Ameriprise Financial,Financials
BAC,Bank of America Corp,Financials
STT,State Street Corp.,Financials
MS,Morgan Stanley,Financials
JPM,JPMorgan Chase & Co.,Financials
GS,Goldman Sachs Group,Financials
BK,The Bank of New York Mellon Corp.,Financials
C,Citigroup Inc.,Financials


getStockDetails('FITB', clusterKMeans, firmDF)

In [9]:
getStockDetails('FITB', clusterKMeans, firmDF)

,Name,Sector
Symbol,,
FITB,Fifth Third Bancorp,Financials
RF,Regions Financial Corp.,Financials
ZION,Zions Bancorp,Financials
CMA,Comerica Inc.,Financials
KEY,KeyCorp,Financials
